In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("../..")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import utils.constants as constants

In [ ]:
df = pd.read_csv("resources/data/raw/lending_club_loan_two.csv")

In [ ]:
df["issue_d"].apply(lambda d: d.split("-")[0]).drop_duplicates()

In [ ]:
# Select relevant target values
df = df.loc[
    (df[constants.TARGET_COLUMN] == constants.FULLY_PAID_STATUS) |
    (df[constants.TARGET_COLUMN] == constants.CHARGED_OFF_STATUS)
]
df[constants.TARGET_COLUMN] = df[constants.TARGET_COLUMN].map({
    constants.FULLY_PAID_STATUS: 0,
    constants.CHARGED_OFF_STATUS: 1
})

In [ ]:
df.shape

In [ ]:
df.columns

# Data Description

In [ ]:
df[constants.TARGET_COLUMN].hist()

In [ ]:
SKEWED_COLUMNS = []
COLUMNS_TO_STANDARDIZE = []
COLUMNS_TO_SCALE = []
NON_ETHICAL_COLUMNS = []
IMBALANCED_COLUMNS = []
COLUMNS_TO_ORDINAL_ENCODING = []
STANDARD_CAT_COLUMNS = []
REDUNDANT_COLUMNS = []
COLUMNS_TO_WOE_ENCODE = []
COLUMNS_TO_IMPUTE_MISSING_CATEGORY = []
COLUMNS_TO_DROP_OUTLIERS = []
DATE_COLUMNS_TO_SPLIT = []
COLUMNS_TO_CYCLICAL_ENCODING = []
COLUMNS_TO_BINARIZE = []
COLUMNS_TO_MICE_IMPUTE = []
COLUMNS_TO_IMPUTE_0 = []

In [ ]:
def describe_num_column(col):
    print(f"{col.isna().sum() = }")
    print("----------------")
    print(col.describe())
    print(f"{col.skew() = }")
    print("----------------")
    print(col.drop_duplicates())
    print("----------------")
    col.hist()

def describe_cat_column(col, plot=True):
    print(f"{col.isna().sum() = }")
    print("----------------")
    print(col.describe())
    print("----------------")
    print(col.drop_duplicates())
    print("----------------")
    print(f"{col.nunique()=}")
    print("----------------")
    if plot:
        col.hist()

### `loan_amnt`

In [ ]:
describe_num_column(df["loan_amnt"])

In [ ]:
SKEWED_COLUMNS.append("loan_amnt")
COLUMNS_TO_STANDARDIZE.append("loan_amnt")

### `term`

In [ ]:
describe_cat_column(df["term"])

In [ ]:
COLUMNS_TO_ORDINAL_ENCODING.append("term")
COLUMNS_TO_SCALE.append("term")

### `int_rate`

In [ ]:
describe_num_column(df["int_rate"])

In [ ]:
SKEWED_COLUMNS.append("loan_amnt")
COLUMNS_TO_STANDARDIZE.append("loan_amnt")

### `installment`

In [ ]:
describe_num_column(df["installment"])

In [ ]:
SKEWED_COLUMNS.append("loan_amnt")
COLUMNS_TO_STANDARDIZE.append("loan_amnt")

### `grade` & `sub_grade`

In [ ]:
df[["grade", "sub_grade"]].sort_values("sub_grade").drop_duplicates()

`sub_grade` is variable dependent on `grade`, so for not keeping high cardinality categorical variables I'll drop `sub_grade` column

In [ ]:
REDUNDANT_COLUMNS.append("sub_grade")
COLUMNS_TO_ORDINAL_ENCODING.append("grade")
COLUMNS_TO_SCALE.append("grade")

### `emp_title`

In [ ]:
describe_cat_column(df["emp_title"], plot=False)

In [ ]:
COLUMNS_TO_IMPUTE_MISSING_CATEGORY.append("emp_title")
COLUMNS_TO_WOE_ENCODE.append("emp_title")

### `emp_length`

In [ ]:
plt.xticks(rotation=45)
describe_cat_column(df["emp_length"])

In [ ]:
COLUMNS_TO_MICE_IMPUTE.append("emp_length")
COLUMNS_TO_ORDINAL_ENCODING.append("emp_length")
COLUMNS_TO_SCALE.append("emp_length")

### `home_ownership`

In [ ]:
describe_cat_column(df["home_ownership"])

We may need to merge "None", "Any", "Other", "Own" into one category

In [ ]:
STANDARD_CAT_COLUMNS.append("home_ownership")

### `annual_inc`

In [ ]:
describe_num_column(df["annual_inc"])

In [ ]:
COLUMNS_TO_DROP_OUTLIERS.append("annual_inc")
SKEWED_COLUMNS.append("annual_inc")
COLUMNS_TO_STANDARDIZE.append("annual_inc")

###  `verification_status`

In [ ]:
describe_cat_column(df["verification_status"])

In [ ]:
STANDARD_CAT_COLUMNS.append("verification_status")

### `issue_d`

In [ ]:
plt.xticks(rotation=90)
describe_cat_column(df["issue_d"], plot=False)
df["issue_d_year"] = df["issue_d"].apply(lambda d: d.split("-")[1])
df["issue_d_year"].hist()

In [ ]:
DATE_COLUMNS_TO_SPLIT.append("issue_d")
COLUMNS_TO_SCALE.append("issue_d_year")
COLUMNS_TO_CYCLICAL_ENCODING.append("issue_d_month")

### `purpose`

In [ ]:
plt.xticks(rotation=90)
describe_cat_column(df["purpose"])

In [ ]:
STANDARD_CAT_COLUMNS.append("purpose")

### `title`

In [ ]:
describe_cat_column(df["title"], plot=False)

As for now let's drop this, but in future we may need to reintroduce this column

### `address`

I will drop this column since I don't want model to predict default probability based on somebody's address

In [ ]:
df["term"].drop_duplicates().values

In [ ]:
NON_ETHICAL_COLUMNS.append("zip_code")
NON_ETHICAL_COLUMNS.append("addr_state")

### `dti`

In [ ]:
describe_num_column(df["dti"])

In [ ]:
COLUMNS_TO_DROP_OUTLIERS.append("dti")
SKEWED_COLUMNS.append("dti")
COLUMNS_TO_STANDARDIZE.append("dti")

### `earliest_cr_line`

In [ ]:
plt.xticks(rotation=90)
describe_cat_column(df["earliest_cr_line"], plot=False)
df["earliest_cr_line_year"] = df["earliest_cr_line"].apply(lambda d: d.split("-")[1])
df["earliest_cr_line_year"].hist()

In [ ]:
DATE_COLUMNS_TO_SPLIT.append("earliest_cr_line")
COLUMNS_TO_SCALE.append("earliest_cr_line_year")
COLUMNS_TO_CYCLICAL_ENCODING.append("earliest_cr_line_month")

### `open_acc`

In [ ]:
describe_cat_column(df["open_acc"])

In [ ]:
COLUMNS_TO_DROP_OUTLIERS.append("open_acc")
SKEWED_COLUMNS.append("open_acc")
COLUMNS_TO_STANDARDIZE.append("open_acc")

### `pub_rec`

In [ ]:
describe_num_column(df["pub_rec"])

In [ ]:
COLUMNS_TO_BINARIZE.append("pub_rec")

### `revol_bal`

In [ ]:
describe_num_column(df["revol_bal"])

In [ ]:
COLUMNS_TO_DROP_OUTLIERS.append("open_acc")
SKEWED_COLUMNS.append("open_acc")
COLUMNS_TO_STANDARDIZE.append("open_acc")

### `revol_util`

In [ ]:
describe_num_column(df["revol_util"])

In [ ]:
COLUMNS_TO_MICE_IMPUTE.append("revol_util")
COLUMNS_TO_DROP_OUTLIERS.append("open_acc")
SKEWED_COLUMNS.append("open_acc")
COLUMNS_TO_STANDARDIZE.append("open_acc")

### `total_acc`

In [ ]:
describe_num_column(df["total_acc"])

In [ ]:
COLUMNS_TO_DROP_OUTLIERS.append("open_acc")
SKEWED_COLUMNS.append("open_acc")
COLUMNS_TO_STANDARDIZE.append("open_acc")

### `initial_list_status`

In [ ]:
describe_cat_column(df["initial_list_status"])

In [ ]:
STANDARD_CAT_COLUMNS.append("initial_list_status")

### `application_type`

In [ ]:
describe_cat_column(df["application_type"])

In [ ]:
IMBALANCED_COLUMNS.append("application_type")

### `mort_acc`

In [ ]:
describe_num_column(df["mort_acc"])

In [ ]:
COLUMNS_TO_MICE_IMPUTE.append("mort_acc")
COLUMNS_TO_DROP_OUTLIERS.append("mort_acc")
COLUMNS_TO_STANDARDIZE.append("mort_acc")

### `pub_rec_bankruptcies`

In [ ]:
describe_num_column(df["pub_rec_bankruptcies"])

In [ ]:
COLUMNS_TO_IMPUTE_0.append("pub_rec_bankruptcies")
COLUMNS_TO_BINARIZE.append("pub_rec_bankruptcies")

## Summary

In [ ]:
print(f"{SKEWED_COLUMNS = }")
print(f"{COLUMNS_TO_STANDARDIZE = }")
print(f"{COLUMNS_TO_SCALE = }")
print(f"{NON_ETHICAL_COLUMNS = }")
print(f"{IMBALANCED_COLUMNS = }")
print(f"{COLUMNS_TO_ORDINAL_ENCODING = }")
print(f"{STANDARD_CAT_COLUMNS = }")
print(f"{REDUNDANT_COLUMNS = }")
print(f"{COLUMNS_TO_WOE_ENCODE = }")
print(f"{COLUMNS_TO_IMPUTE_MISSING_CATEGORY = }")
print(f"{COLUMNS_TO_DROP_OUTLIERS = }")
print(f"{DATE_COLUMNS_TO_SPLIT = }")
print(f"{COLUMNS_TO_CYCLICAL_ENCODING = }")
print(f"{COLUMNS_TO_BINARIZE = }")
print(f"{COLUMNS_TO_MICE_IMPUTE = }")
print(f"{COLUMNS_TO_IMPUTE_0 = }")

# Missing Values

In [ ]:
df.isna().any(axis=0)

## Correlations

In [ ]:
sns.heatmap(df.corr(method="spearman"), cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
COLUMNS_HIGHLY_CORRELATED_TO_DROP=["installment"]